# Downloading & Importing Libraries

In [1]:
!pip install --upgrade wandb

!pip install efficientnet

     |████████████████████████████████| 1.6 MB 1.4 MB/s 
  Attempting uninstall: wandb
    Found existing installation: wandb 0.9.4
    Uninstalling wandb-0.9.4:
      Successfully uninstalled wandb-0.9.4
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
     |████████████████████████████████| 50 kB 1.2 MB/s 
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf

import matplotlib.pyplot as plt
from kaggle_datasets import KaggleDatasets
from sklearn.model_selection import train_test_split

import efficientnet.tfkeras as efn

import wandb
from wandb.keras import WandbCallback

In [3]:
tf.__version__

'2.2.0'

In [4]:
!wandb login c1791ecafb99a7335843177aeef2935db9600e4b

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# Initialize TPU

In [5]:
# Detect TPU

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print("Running on TPU", tpu.master())
    
except:
    tpu = None
    print("Couldn't able to Find TPU :(")
    
if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()
    
    
print("Replicas :", strategy.num_replicas_in_sync)

# Data Access from GCS

GCS_PATH = KaggleDatasets().get_gcs_path()
print(GCS_PATH)

Running on TPU grpc://10.0.0.2:8470
Replicas : 8
gs://kds-9ec1742d9fc84e86a4d160af664dcc2778b6b754c90c3122654cf571


# Imporing Dataset

In [6]:
train = pd.read_csv("../input/plant-pathology-2020-fgvc7/train.csv")

train

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0
...,...,...,...,...,...
1816,Train_1816,0,0,0,1
1817,Train_1817,1,0,0,0
1818,Train_1818,1,0,0,0
1819,Train_1819,0,0,1,0


In [7]:
train['image_id'] = GCS_PATH + '/images/' + train['image_id'] +'.jpg'
train

,image_id,healthy,multiple_diseases,rust,scab
0,gs://kds-9ec1742d9fc84e86a4d160af664dcc2778b6b...,0,0,0,1
1,gs://kds-9ec1742d9fc84e86a4d160af664dcc2778b6b...,0,1,0,0
2,gs://kds-9ec1742d9fc84e86a4d160af664dcc2778b6b...,1,0,0,0
3,gs://kds-9ec1742d9fc84e86a4d160af664dcc2778b6b...,0,0,1,0
4,gs://kds-9ec1742d9fc84e86a4d160af664dcc2778b6b...,1,0,0,0
...,...,...,...,...,...
1816,gs://kds-9ec1742d9fc84e86a4d160af664dcc2778b6b...,0,0,0,1
1817,gs://kds-9ec1742d9fc84e86a4d160af664dcc2778b6b...,1,0,0,0
1818,gs://kds-9ec1742d9fc84e86a4d160af664dcc2778b6b...,1,0,0,0
1819,gs://kds-9ec1742d9fc84e86a4d160af664dcc2778b6b...,0,0,1,0


In [8]:
train['image_id'][0]

'gs://kds-9ec1742d9fc84e86a4d160af664dcc2778b6b754c90c3122654cf571/images/Train_0.jpg'

In [9]:
img_paths = train['image_id'].values
labels = train.iloc[:, 1:].values

img_paths, labels

(array(['gs://kds-9ec1742d9fc84e86a4d160af664dcc2778b6b754c90c3122654cf571/images/Train_0.jpg',
        'gs://kds-9ec1742d9fc84e86a4d160af664dcc2778b6b754c90c3122654cf571/images/Train_1.jpg',
        'gs://kds-9ec1742d9fc84e86a4d160af664dcc2778b6b754c90c3122654cf571/images/Train_2.jpg',
        ...,
        'gs://kds-9ec1742d9fc84e86a4d160af664dcc2778b6b754c90c3122654cf571/images/Train_1818.jpg',
        'gs://kds-9ec1742d9fc84e86a4d160af664dcc2778b6b754c90c3122654cf571/images/Train_1819.jpg',
        'gs://kds-9ec1742d9fc84e86a4d160af664dcc2778b6b754c90c3122654cf571/images/Train_1820.jpg'],
       dtype=object),
 array([[0, 0, 0, 1],
        [0, 1, 0, 0],
        [1, 0, 0, 0],
        ...,
        [1, 0, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]]))

In [10]:
train_img, valid_img, train_labels, valid_labels = train_test_split(img_paths, labels, test_size=0.1)

# Making Training & Validation datasets

In [11]:
# The function to take a imahe path and return the image ( in array )
def read_img(filename, label=None, img_size=(512, 512)):
    
    bits = tf.io.read_file(filename)
    
    img = tf.image.decode_jpeg(bits, channels=3)
    
    img = tf.cast(img, tf.float32) / 255.0
    
    img = tf.image.resize(img, img_size)
    
    if label is None:
        return img
    
    else:
        return img, label

# The function to do the data argumentation
def data_aug(img, label=None):
    
    img = tf.image.random_flip_left_right(img)
    
    img = tf.image.random_flip_up_down(img)
    
    if label is None:
        return img
    else:
        return img, label

In [12]:
AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((train_img, train_labels))
    .map(read_img, num_parallel_calls=AUTO)
    .cache()
    .map(data_aug, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO))

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((valid_img, valid_labels))
    .map(read_img, num_parallel_calls=AUTO)
    .cache()
    .map(data_aug, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO))

In [13]:
train_dataset

<PrefetchDataset shapes: ((None, 512, 512, 3), (None, 4)), types: (tf.float32, tf.int64)>

# Creating Our Model

In [14]:
def create_model():
    return tf.keras.models.Sequential([
        efn.EfficientNetB7(include_top=False, weights='imagenet', input_shape=(512, 512, 3)),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(4, activation='softmax')
    ])

In [15]:
# with strategy.scope(): # to use TPU while training
#     model = create_model()
#     model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC(name='AUC')])
#     model.summary()

In [16]:
# # Initlisazie wandb project
# wandb.init(project='plant disease classification',
#           notes='first tpu training & using efficient net b7',
#           name='first-tpu-training')

In [17]:
# Making the wandb callback
wandb_callback = WandbCallback()

Error: You must call wandb.init() before WandbCallback()

In [18]:
EPOCHS = 20

In [19]:
# model.fit(train_dataset, epochs=EPOCHS, 
#           validation_data=valid_dataset, 
#           callbacks=[wandb_callback])

# Creating & Training Ensemble Models

In [20]:
# Creating our ensemble Model
def create_ensemble_model():
    
    efficientnetb7 = tf.keras.models.Sequential([
        efn.EfficientNetB7(include_top=False, weights='imagenet', input_shape=(512, 512, 3)),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(4, activation='softmax')
    ])
    
    #efficientnetb7.layers[0].trainable = False
    
    xception = tf.keras.models.Sequential([
        tf.keras.applications.xception.Xception(include_top=False, weights='imagenet', input_shape=(512, 512, 3)),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(4, activation='softmax')
        
    ])
    
    #xception.layers[0].trainable = False
    
    inputs = tf.keras.Input(shape=(512, 512, 3))
    
    efficientnetb7_output = efficientnetb7(inputs)
    xception_output = xception(inputs)
    
    outputs = tf.keras.layers.average([efficientnetb7_output, xception_output])
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    return model

In [21]:
# with strategy.scope():
#     model = create_ensemble_model()
#     model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC(name='AUC')])
#     model.summary()

In [22]:
# wandb.init(project='plant disease classification',
#           name='tpu training ensemble',
#           notes='Training our first ensemble model using TPU')

In [23]:
wandb_callback = WandbCallback()

Error: You must call wandb.init() before WandbCallback()

In [24]:
# model.fit(train_dataset, epochs=20, validation_data=valid_dataset, callbacks=[wandb_callback])

# Hyperparameter Tuning

In [25]:
optimizers = ['adam', 'sgd']
weights = ['imagenet', None]

In [26]:
def create_model(weight):
    return tf.keras.models.Sequential([
        efn.EfficientNetB7(include_top=False, weights=weight, input_shape=(512, 512, 3)),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(4, activation='softmax')])

In [27]:
# for optimizer in optimizers:
    
#     for weight in weights:
        
#         with strategy.scope():
        
#             model = create_model(weight)
            
#             model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC(name='AUC')])
            
#             model.summary()
            
        
#         config = {'optimizer':optimizer, 'weight':weight}
        
#         print(config)
        
#         wandb.init(project='plant disease classification', config=config, name=f'hyper-tune-optm-{optimizer}-weight-{weight}')
        
#         wandb_callback = WandbCallback()
        
#         model.fit(train_dataset, epochs=20, validation_data=valid_dataset, callbacks=[wandb_callback])
        
        
#         tf.tpu.experimental.initialize_tpu_system(tpu)

# BIG Hyperparameter Tuning

In [28]:
learning_rate = [0.0001, 0.005, 0.05, 0.1, 0.3] # 0.001 - Default LR

In [29]:
def create_model():
    return tf.keras.models.Sequential([
        efn.EfficientNetB7(include_top=False, weights='imagenet', input_shape=(512, 512, 3)),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(4, activation='softmax')
    ])

In [30]:
for lr in learning_rate:
    
    with strategy.scope():
        
        model = create_model()
        
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC(name='AUC')])
        
    config = {'learning rate':lr}
    
    print(config)
    
    wandb.init(project='plant disease classification', config=config, name=f'hyper-tune-lr-{lr}')
    
    wandb_callback = WandbCallback(save_model=False)
    
    model.fit(train_dataset, epochs=30, validation_data=valid_dataset, callbacks=[wandb_callback])
    
    tf.tpu.experimental.initialize_tpu_system(tpu)

258441216/258434480 [==============================] - 4s 0us/step
{'learning rate': 0.0001}


wandb: Currently logged in as: shubhamai (use `wandb login --relogin` to force relogin)
wandb: Tracking run with wandb version 0.10.2
wandb: Run data is saved locally in wandb/run-20200927_170554-kgyx2j2e
wandb: Syncing run hyper-tune-lr-0.0001



Epoch 1/30
13/13 [==============================] - 192s 15s/step - loss: 1.1830 - AUC: 0.8032 - accuracy: 0.5690 - val_loss: 1.1292 - val_AUC: 0.8430 - val_accuracy: 0.6011
Epoch 2/30
13/13 [==============================] - 12s 952ms/step - loss: 0.5643 - AUC: 0.9598 - accuracy: 0.8620 - val_loss: 0.6271 - val_AUC: 0.9341 - val_accuracy: 0.7541
Epoch 3/30
13/13 [==============================] - 13s 994ms/step - loss: 0.2779 - AUC: 0.9849 - accuracy: 0.9164 - val_loss: 0.6499 - val_AUC: 0.9398 - val_accuracy: 0.7814
Epoch 4/30
13/13 [==============================] - 12s 912ms/step - loss: 0.2150 - AUC: 0.9901 - accuracy: 0.9280 - val_loss: 0.5819 - val_AUC: 0.9486 - val_accuracy: 0.7923
Epoch 5/30
13/13 [==============================] - 12s 917ms/step - loss: 0.1648 - AUC: 0.9948 - accuracy: 0.9481 - val_loss: 0.4348 - val_AUC: 0.9685 - val_accuracy: 0.8361
Epoch 6/30
13/13 [==============================] - 12s 922ms/step - loss: 0.1452 - AUC: 0.9959 - accuracy: 0.9548 - val_loss

wandb: Waiting for W&B process to finish, PID 179
wandb: Program ended successfully.


{'learning rate': 0.005}



...


Epoch 1/30
13/13 [==============================] - 180s 14s/step - loss: 15.6128 - AUC: 0.5201 - accuracy: 0.2601 - val_loss: nan - val_AUC: 0.0000e+00 - val_accuracy: 0.3224
Epoch 2/30
13/13 [==============================] - 12s 924ms/step - loss: 3.7534 - AUC: 0.6020 - accuracy: 0.3315 - val_loss: nan - val_AUC: 0.0000e+00 - val_accuracy: 0.3224
Epoch 3/30
13/13 [==============================] - 12s 918ms/step - loss: 2.3142 - AUC: 0.6094 - accuracy: 0.2918 - val_loss: 55588875884498185894232064.0000 - val_AUC: 0.3552 - val_accuracy: 0.0328
Epoch 4/30
13/13 [==============================] - 12s 918ms/step - loss: 1.9255 - AUC: 0.6120 - accuracy: 0.3077 - val_loss: 293691238090014720.0000 - val_AUC: 0.3552 - val_accuracy: 0.0328
Epoch 5/30
13/13 [==============================] - 12s 919ms/step - loss: 1.8030 - AUC: 0.6104 - accuracy: 0.3144 - val_loss: 3276275449856.0000 - val_AUC: 0.3552 - val_accuracy: 0.0328
Epoch 6/30
13/13 [==============================] - 12s 921ms/step -

wandb: Waiting for W&B process to finish, PID 260
wandb: Program ended successfully.


{'learning rate': 0.3}



wandb:                                                                                
wandb: Find user logs for this run at: wandb/run-20200927_175109-2vw4ai3b/logs/debug.log
wandb: Find internal logs for this run at: wandb/run-20200927_175109-2vw4ai3b/logs/debug-internal.log
wandb: Run summary:
wandb:           epoch 29
wandb:            loss 1.164654016494751
wandb:             AUC 0.7478012442588806
wandb:        accuracy 0.4670329689979553
wandb:        val_loss 1.1257728338241577
wandb:         val_AUC 0.7701732516288757
wandb:    val_accuracy 0.5245901942253113
wandb:           _step 29
wandb:        _runtime 830
wandb:      _timestamp 1601229903
wandb:   best_val_loss 1.1257728338241577
wandb:      best_epoch 29
wandb: Run history:
wandb:          epoch ▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
wandb:           loss █▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:            AUC ▁▄▄▄▄▄▅▅▄▅▅▅▅▄▅▅▆▆▇▇▇▇▇▇██████
wandb:       accuracy ▁▃▂▃▃▃▃▃▃▃▄▄▃▃▃▄▄▆▆▅▅▆▇▇▇▇█▇▇█
wandb:       val_loss █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


Epoch 1/30
13/13 [==============================] - 176s 14s/step - loss: 67.1652 - AUC: 0.5083 - accuracy: 0.2674 - val_loss: nan - val_AUC: 0.0000e+00 - val_accuracy: 0.3224
Epoch 2/30
13/13 [==============================] - 12s 930ms/step - loss: 13.4623 - AUC: 0.5615 - accuracy: 0.3175 - val_loss: nan - val_AUC: 0.0000e+00 - val_accuracy: 0.3224
Epoch 3/30
13/13 [==============================] - 12s 924ms/step - loss: 11.0441 - AUC: 0.5347 - accuracy: 0.2863 - val_loss: nan - val_AUC: 0.0000e+00 - val_accuracy: 0.3224
Epoch 4/30
13/13 [==============================] - 12s 926ms/step - loss: 11.2913 - AUC: 0.5461 - accuracy: 0.2961 - val_loss: 50339665779197377458995200.0000 - val_AUC: 0.3552 - val_accuracy: 0.0328
Epoch 5/30
13/13 [==============================] - 12s 921ms/step - loss: 6.7698 - AUC: 0.5781 - accuracy: 0.3248 - val_loss: 1032766667765055488.0000 - val_AUC: 0.3552 - val_accuracy: 0.0328
Epoch 6/30
13/13 [==============================] - 12s 920ms/step - loss: 